In [ ]:
! pip -q install --upgrade together
! pip install langchain
! pip install python-decouple
! pip install "pydantic==1.*"
! pip install transformers
!pip install -q gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.9 MB/s eta 

In [ ]:
from decouple import config
import together

together.api_key='8e0125f2beb859507e6bcd5fa13ca51d457acb65d9132879bda89944a6e72481'


In [ ]:
models=together.Models.list()


for i, model in enumerate(models):
  print(i,model['name'])

0 Austism/chronos-hermes-13b
1 EleutherAI/llemma_7b
2 EleutherAI/pythia-12b-v0
3 EleutherAI/pythia-1b-v0
4 EleutherAI/pythia-2.8b-v0
5 EleutherAI/pythia-6.9b
6 Gryphe/MythoMax-L2-13b
7 HuggingFaceH4/starchat-alpha
8 NousResearch/Nous-Hermes-13b
9 NousResearch/Nous-Hermes-Llama2-13b
10 NousResearch/Nous-Hermes-Llama2-70b
11 NousResearch/Nous-Hermes-llama-2-7b
12 NumbersStation/nsql-llama-2-7B
13 Open-Orca/Mistral-7B-OpenOrca
14 OpenAssistant/oasst-sft-4-pythia-12b-epoch-3.5
15 OpenAssistant/stablelm-7b-sft-v7-epoch-3
16 Phind/Phind-CodeLlama-34B-Python-v1
17 Phind/Phind-CodeLlama-34B-v2
18 SG161222/Realistic_Vision_V3.0_VAE
19 WizardLM/WizardCoder-15B-V1.0
20 WizardLM/WizardLM-70B-V1.0
21 bigcode/starcoder
22 databricks/dolly-v2-3b
23 databricks/dolly-v2-7b
24 defog/sqlcoder
25 garage-bAInd/Platypus2-70B-instruct
26 huggyllama/llama-13b
27 huggyllama/llama-30b
28 huggyllama/llama-65b
29 huggyllama/llama-7b
30 lmsys/fastchat-t5-3b-v1.0
31 lmsys/vicuna-13b-v1.5-16k
32 lmsys/vicuna-13b-v1.

In [ ]:
together.Models.start("togethercomputer/llama-2-7b-chat")

{'success': True,
 'value': '8fadd0868b5629472202aee7a8e5a64867dd21cd870fff0f20cea38d795732aa-38affa63e0e6842a8ec7da640b9281d3a7ca8c8476c0f6499a54ac3a9a3b64c2'}

In [ ]:
from typing import Any,Dict
from pydantic import Extra, root_validator
from langchain.llms.base import LLM
from langchain.utils import get_from_dict_or_env

TG_apikey='8e0125f2beb859507e6bcd5fa13ca51d457acb65d9132879bda89944a6e72481'

class TogetherLLM(LLM):
  model="togethercomputer/llama-2-7b-chat"

  together_api_key= TG_apikey

  temperature =0.3

  max_tokens=256

  class Config:
    extra = Extra.forbid

  @root_validator(skip_on_failure=True)

  def validate_environment(cls,values:Dict)-> Dict:
    api_key=get_from_dict_or_env(
        values,"together_api_key","TG_apikey"
    )
    values["together_api_key"]=api_key
    return values

  @property
  def _llm_type(self):
    return "together"

  def _call(
      self,
      prompt,
      **kwargs:Any,
  ):
    together.api_key = self.together_api_key
    output = together.Complete.create(prompt,
                                      model=self.model,
                                      max_tokens=self.max_tokens,
                                      temperature=self.temperature,)
    text=output['output']['choices'][0]['text']
    return text

<ipython-input-8-1b0e802465eb>:18: PydanticDeprecatedSince20: `pydantic.config.Extra` is deprecated, use literal values instead (e.g. `extra='allow'`). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  extra = Extra.forbid
<ipython-input-8-1b0e802465eb>:20: PydanticDeprecatedSince20: Pydantic V1 style `@root_validator` validators are deprecated. You should migrate to Pydantic V2 style `@model_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  @root_validator(skip_on_failure=True)


TypeError: ignored

In [ ]:
tog_llm=TogetherLLM(
    model="togethercomputer/llama-2-7b-chat",
    temperature=0.2,
    max_tokens=256,
)

In [ ]:
B_INST, E_INST= "[INST]", "[/INST]"

B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

DEFAULT_SYSTEM_PROMPT ="""You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent,
explain why instead of answering something not correct.
If you don't know the answer to a question, please don't share false information."""

def get_prompt(instruction,new_system_prompt=DEFAULT_SYSTEM_PROMPT):
# def get_prompt(instruction,input_text='',new_system_prompt=DEFAULT_SYSTEM_PROMPT):

  SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
  prompt_template = B_INST + SYSTEM_PROMPT + instruction + E_INST
  return prompt_template

In [ ]:
system_prompt="You are an expert pediatric doctor who excels in understand problems that kids deal with."
instruction="Answer the problems or queries that people ask. Understand the possible problems that they have, but do not share what it is.Then give solutions to the problems they are dealing with.\n\n{text}"
template1=get_prompt(instruction,system_prompt)

In [ ]:
print(template1)

In [ ]:
from langchain import PromptTemplate,LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain import PromptTemplate,LLMChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


# from langchain.prompts import PromptTemplate
# from langchain.chains import ConversationChain
# from langchain.llm import LLM

# Define your prompt template
prompt_template = PromptTemplate(
    input_variables=["history", "input"],
    template="You are an expert pediatric assistant who excels in understand problems that kids deal with. If anything is unclear, ask back questions to avoid ambiguity. Always end the response on a supportive note and give recommendations for the problems they are facing.  \nCurrent conversation: {history} \nHuman: {input} \nAI Assistant:"
)

# Define your language model (replace 'your-model' with the actual model name)
# llm = LLM(model_name='your-model')

# Create your ConversationChain
# conversation = ConversationChain(
#     prompt=prompt_template,
#     llm=tog_llm,
#     verbose=True,
# )



# prompt_template = ChatPromptTemplate.from_template(template)

# print(template)

In [ ]:
# prompt=PromptTemplate(template=template,input_variables=["text"])
memory = ConversationBufferMemory(llm=tog_llm, max_token_limit=512)

In [ ]:
conversation = ConversationChain(
    llm=tog_llm,
    memory = memory,
    verbose=True,
    prompt=prompt_template,

    )

In [ ]:
type(memory.load_memory_variables({}))

In [ ]:
conversation.predict(input="Hi my name is nirmal.")

Demo

In [ ]:
# print(memory.load_memory_variables({})['history'])
memory.load_memory_variables({})

In [ ]:
# ! pip install typing-extensions --upgrade


In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    # clear = gr.Button(value="Clear")
    clear = gr.ClearButton([msg, chatbot])

    # gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = conversation.predict(input=message)
        chat_history.append((message, bot_message))
        # time.sleep(2)
        return "", chat_history

    def on_clear_click():
          print("CLEAR BUTTON CLICKED!")
          # gr.ClearButton([msg, chatbot])
          memory.clear()
          # memory.load_memory_variables({})['history']=''
          # print(memory.buffer)

    clear.click(on_clear_click)

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    # if clear.click():
    #   on_clear_click()
demo.launch()

In [ ]:
# memory.clear()
print(memory.load_memory_variables({})['history'])

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = conversation.predict(input=message)
        chat_history.append((message, bot_message))
        # time.sleep(2)
        return "", chat_history

    def on_clear_click():
          print("CLEAR BUTTON CLICKED!")
          memory.clear()
          # memory.load_memory_variables({})['history']=''
          # print(memory.buffer)
    clear.click(on_clear_click)

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    # if clear.click():
    #   on_clear_click()
demo.launch()


In [ ]:
# memory.clear()
print(memory.load_memory_variables({})['history'])

In [ ]:
text="My friend injured his knee while he was squading, his knees hurt. He has little mobility. What should I do to help him?"
# prompt = prompt_template.format_messages(text)

# print(prompt)

In [ ]:
conversation.predict(input=text)



> Entering new ConversationChain chain...
Prompt after formatting:
You are an expert pediatric assistant who excels in understand problems that kids deal with. 
Current conversation:  
Human: My friend injured his knee while he was squading, his knees hurt. He has little mobility. What should I do to help him? 
AI Assistant:

> Finished chain.


"I'm so sorry to hear that your friend is dealing with a knee injury! As a pediatric assistant, I understand how difficult it can be for kids to deal with pain and limited mobility. Here are some things you can do to help your friend:\n\n1. Encourage them to rest: It's important to give your friend's knee a break and let it heal. Encourage them to rest and avoid any activities that may exacerbate the injury.\n2. Apply ice: Ice can help reduce swelling and relieve pain. Have your friend apply ice to their knee for 15-20 minutes at a time, several times a day.\n3. Use compression: Wrapping your friend's knee with an elastic bandage can help reduce swelling. Be sure to wrap it snugly but not too tightly, as too much compression can cut off circulation.\n4. Elevate: Elevating your friend's knee above the level of their heart can help reduce swelling and improve circulation.\n5. Offer pain relief: Over-the-counter pain medications such as acetaminophen"

In [ ]:
print(conversation.predict(input="now theres swelling and redness. No mobility and extreme pain. What should be done?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are an expert pediatric assistant who excels in understand problems that kids deal with. 
Current conversation: Human: My friend injured his knee while he was squading, his knees hurt. He has little mobility. What should I do to help him?
AI: I'm so sorry to hear that your friend is dealing with a knee injury! As a pediatric assistant, I understand how difficult it can be for kids to deal with pain and limited mobility. Here are some things you can do to help your friend:

1. Encourage them to rest: It's important to give your friend's knee a break and let it heal. Encourage them to rest and avoid any activities that may exacerbate the injury.
2. Apply ice: Ice can help reduce swelling and relieve pain. Have your friend apply ice to their knee for 15-20 minutes at a time, several times a day.
3. Use compression: Wrapping your friend's knee with an elastic bandage can help reduce swelling. Be sure to wrap it snugly

In [ ]:
memory.buffer

"Human: My friend injured his knee while he was squading, his knees hurt. He has little mobility. What should I do to help him?\nAI: I'm so sorry to hear that your friend is dealing with a knee injury! As a pediatric assistant, I understand how difficult it can be for kids to deal with pain and limited mobility. Here are some things you can do to help your friend:\n\n1. Encourage them to rest: It's important to give your friend's knee a break and let it heal. Encourage them to rest and avoid any activities that may exacerbate the injury.\n2. Apply ice: Ice can help reduce swelling and relieve pain. Have your friend apply ice to their knee for 15-20 minutes at a time, several times a day.\n3. Use compression: Wrapping your friend's knee with an elastic bandage can help reduce swelling. Be sure to wrap it snugly but not too tightly, as too much compression can cut off circulation.\n4. Elevate: Elevating your friend's knee above the level of their heart can help reduce swelling and improv

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        bot_message = conversation.predict(message)
        chat_history.append((message, bot_message))
        # time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Using chains

In [ ]:
template=get_prompt(instruction,system_prompt)

In [ ]:
print(template)

[INST]<<SYS>>
You are an expert pediatric doctor who excels in understand problems that kids deal with.
<</SYS>>

Answer the problems or queries that people ask. Understand the possible problems that they have, but do not share what it is.Then give solutions to the problems they are dealing with.

{text}[/INST]


In [ ]:
prompt = ChatPromptTemplate.from_template(template)
print(prompt)

input_variables=['text'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], template='[INST]<<SYS>>\nYou are an expert pediatric doctor who excels in understand problems that kids deal with.\n<</SYS>>\n\nAnswer the problems or queries that people ask. Understand the possible problems that they have, but do not share what it is.Then give solutions to the problems they are dealing with.\n\n{text}[/INST]'))]


In [ ]:
# prompt template 1
memory = ConversationSummaryBufferMemory(llm=tog_llm, max_token_limit=256)

# Chain 1
chain = LLMChain(llm=tog_llm, prompt=prompt,memory=memory)
text = "My injured his ankle few days back, he is having difficulty with moving his leg. What should be done?"
chain.run(text)

" Ah, I see. Well, it's important to assess the severity of the injury and determine the best course of action. Can you tell me more about the injury? How did it happen? And how long ago did it occur? This information will help me provide you with more tailored advice.\n\nIn the meantime, it's important to keep an eye on your child's symptoms and watch for any signs of worsening. If you notice any significant changes or if your child is experiencing severe pain, you should seek medical attention immediately.\n\nIn the meantime, there are some things you can do to help your child manage their discomfort. For example, you can try applying ice to the affected area to reduce swelling and pain. You can also elevate their leg to reduce swelling and promote healing.\n\nIt's also important to encourage your child to rest and avoid putting too much weight on their injured ankle. This will help the healing process and prevent further injury.\n\nIf you have any concerns or if your child's symptom

In [ ]:
memory.load_memory_variables({})

{'history': "Human: My injured his ankle few days back, he is having difficulty with moving his leg. What should be done?\nAI:  Ah, I see. Well, it's important to assess the severity of the injury and determine the best course of action. Can you tell me more about the injury? How did it happen? And how long ago did it occur? This information will help me provide you with more tailored advice.\n\nIn the meantime, it's important to keep an eye on your child's symptoms and watch for any signs of worsening. If you notice any significant changes or if your child is experiencing severe pain, you should seek medical attention immediately.\n\nIn the meantime, there are some things you can do to help your child manage their discomfort. For example, you can try applying ice to the affected area to reduce swelling and pain. You can also elevate their leg to reduce swelling and promote healing.\n\nIt's also important to encourage your child to rest and avoid putting too much weight on their injure

In [ ]:
text = "He now has swelling and very little mobility of his leg. What do you think is going on now?"
chain.run(text)

' As a pediatric doctor, I understand that the swelling and limited mobility of the leg in a child can be a cause for concern. There are several possible reasons why this may be occurring, including:\n\n1. Injury or trauma: The child may have injured their leg through a fall or a direct blow, leading to swelling and limited mobility.\n2. Infection: A bacterial or viral infection can cause swelling and redness in the leg, which can lead to limited mobility.\n3. Blood clot: A blood clot can form in the leg, causing swelling and pain.\n4. Autoimmune disorder: Certain autoimmune disorders, such as juvenile idiopathic arthritis, can cause swelling and limited mobility in the leg.\n5. Congenital conditions: Some congenital conditions, such as clubfoot or cerebral palsy, can cause swelling and limited mobility in the leg.\n\nIt is important to have the child evaluated by a medical professional to determine the underlying cause of the swelling and limited mobility. A thorough physical examinat

In [ ]:
memory.load_memory_variables({})

{'history': "System: \nThe human asks for advice on how to help their injured ankle. The AI provides information on how to assess the severity of the injury, watch for signs of worsening, and manage discomfort. The AI also encourages the human to seek medical attention if they have concerns or if their child's symptoms worsen.\nEND OF EXAMPLE\nHuman: He now has swelling and very little mobility of his leg. What do you think is going on now?\nAI:  As a pediatric doctor, I understand that the swelling and limited mobility of the leg in a child can be a cause for concern. There are several possible reasons why this may be occurring, including:\n\n1. Injury or trauma: The child may have injured their leg through a fall or a direct blow, leading to swelling and limited mobility.\n2. Infection: A bacterial or viral infection can cause swelling and redness in the leg, which can lead to limited mobility.\n3. Blood clot: A blood clot can form in the leg, causing swelling and pain.\n4. Autoimmun